# Download YouTube Audio Library

### 相關安裝
* pip install selenium
* pip install requests
* 需要下載chromedriver
    * [windows](http://chromedriver.chromium.org/downloads)
    * linux
        * `wget https://chromedriver.storage.googleapis.com/74.0.3729.6/chromedriver_linux64.zip` 
        * `unzip chromedriver_linux64.zip`

* [Creating and Deleting Directories with Python](https://stackabuse.com/creating-and-deleting-directories-with-python/)
* [Python selenium下載檔案至指定路徑](https://huilansame.github.io/huilansame.github.io/archivers/how-to-download-files-with-selenium)
* [JavaScript Fetch API and using Async/Await](https://dev.to/shoupn/javascript-fetch-api-and-using-asyncawait-47mp)
* [Download Returned Zip file from URL](https://stackoverflow.com/questions/9419162/download-returned-zip-file-from-url)

In [1]:
from selenium import webdriver  #從library中引入webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import sys
import os
import io
import getpass
import zipfile
import requests
import json
import pandas as pd
import threading

In [2]:
filepath = "./chromedriver.exe"

if os.path.isfile(filepath):
    print("existed")
else:
    # for windows - version 74
    url = 'https://chromedriver.storage.googleapis.com/74.0.3729.6/chromedriver_win32.zip'
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()  

existed


In [3]:
class YouTubeAudio(object):
    def __init__(self):
        # download directory of json file
        self.json_dir = self.__createDirectory("YouTube-Audio-Library/json")
        self.audio_dir = self.__createDirectory("YouTube-Audio-Library/audio")
    
    def __createDriver(self):
        # option
        self.options = webdriver.ChromeOptions()
        self.prefs = {
            'profile.default_content_settings.popups': 0,
            'profile.default_content_setting_values.automatic_downloads': 1,
            'download.default_directory': self.json_dir
        }
        self.options.add_experimental_option('prefs', self.prefs)
        
        # driver
        self.driver = webdriver.Chrome(options = self.options)        

    def login(self):
        self.__createDriver()
        self.__printMessage("If you want to download audio data, you must login to the google account.")
        self.__printMessage("So...")
        self.__printMessage("Please input your account and password!")
        
        account = input("Account:")
        password = getpass.getpass('Password:')
        
        # YouTube Audio Library
        self.driver.get("https://www.youtube.com/audiolibrary/music")

        self.driver.implicitly_wait(3)
        self.driver.find_element_by_id("identifierId").send_keys(account)
        self.driver.find_element_by_id("identifierNext").click()

        self.driver.implicitly_wait(3)
        password_element = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@type='password']")))
        password_element = WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//input[@type='password']")))

        password_element = self.driver.find_element_by_xpath("/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div[2]/div/div[1]/div/form/content/section/div/content/div[1]/div/div[1]/div/div[1]/input")
        password_element.send_keys(password)
        

        element = self.driver.find_element_by_id('passwordNext')
        self.driver.execute_script("arguments[0].click();", element)
        
        account = None
        password = None
        
        time.sleep(5)

    def __printMessage(self, string):
        for i in range(len(string)):
            print(string[i], end='')
            time.sleep(0.025) 
        print("\n")
        
    def __createDirectory(self, path):
        base_dir = os.getcwd()

        try:  
            os.makedirs(path)
        except OSError:  
            print ("Creation of the directory %s failed" % path)
        else:  
            print ("Successfully created the directory %s" % path)      
            
        return base_dir + "\\" + path
    
    def downloadJSON(self):      
        self.driver.execute_script("(()=>{\
            function saveToFile(filename, data) {\
                let pom = document.createElement('a');\
                pom.setAttribute('href', 'data:text/plain;charset=utf-8,' + encodeURIComponent(data));\
                pom.setAttribute('download', filename);\
                if (document.createEvent) {\
                    let event = document.createEvent('MouseEvents');\
                    event.initEvent('click', true, true);\
                    pom.dispatchEvent(event);\
                }\
                else {\
                    pom.click();\
                }\
            }\
            async function callAPI(index) \
            {\
                let response = await fetch('https://www.youtube.com/audioswap_ajax?action_get_tracks=1&ads=false&dl=true&s=music&mr=25&si=' + index*25 + '&qid=' + index + '&sh=true');\
                let data = await response.json();\
                return data;\
            }\
            function downloadAll(index=0) {\
                callAPI(index).then((data)=>{\
                    console.log(data);\
                    if(data.has_more) {\
                        saveToFile('json_' + index + '.json', JSON.stringify(data));\
                        index++;\
                        downloadAll(index);\
                    }\
                    else{\
                        console.log('no file');\
                    }\
                })\
            }\
            downloadAll();\
            })();\
        ")
        
    def downloadMP3(self):
        index = 0
        json_list = os.listdir(self.json_dir)
        print(len(json_list))
        for i in json_list:
            df = pd.read_json(self.json_dir + "/" + i, encoding='utf-8')
            for j in range(len(df)):
                url = df.iloc[j]['tracks']['download_url']
                filename = youtubeAudio.audio_dir + "/audio_" + str(index) + ".mp3" 
                
                if os.path.isfile(filename):
                    print('existed')
                else:       
                    # requests
                    r = requests.get(url)
                    with open(filename, 'wb') as f:  
                        f.write(r.content)    
                index+=1

        
# # 以下方式會下載不完全、因為window.myJson被覆蓋掉。
#     def query(self):
# #         self.driver.execute_script("console.log('Hello~javascript~');")   
# #         self.driver.execute_script("(()=>{console.log('hello~~~~');})();")
#         # request
        
#         self.driver.execute_script("(()=>{\
#             function download(filename, text) {\
#                 var pom = document.createElement('a');\
#                 pom.setAttribute('href', 'data:text/plain;charset=utf-8,' + encodeURIComponent(text));\
#                 pom.setAttribute('download', filename);\
#                 if (document.createEvent) {\
#                     var event = document.createEvent('MouseEvents');\
#                     event.initEvent('click', true, true);\
#                     pom.dispatchEvent(event);\
#                 }\
#                 else {\
#                     pom.click();\
#                 }\
#             }\
#             for(let i = 0; i < 170; i++) {\
#                 fetch('https://www.youtube.com/audioswap_ajax?action_get_tracks=1&ads=false&dl=true&s=music&mr=25&si='+ i*25 +'&qid='+ i +'&sh=true')\
#                   .then(function(response) {\
#                     return response.json();\
#                   })\
#                   .then(function(myJson) {\
#                     console.log(myJson);\
#                     window.data = myJson;\
#                     download('test_' + i + '.json', JSON.stringify(myJson))\
#                   });\
#             }\
#            })();\
#         ")  

In [4]:
youtubeAudio = YouTubeAudio()

Creation of the directory YouTube-Audio-Library/json failed
Creation of the directory YouTube-Audio-Library/audio failed


In [5]:
youtubeAudio.login()

If you want to download audio data, you must login to the google account.

So...

Please input your account and password!

Account:lemon84672776@gmail.com
Password:········


In [6]:
youtubeAudio.downloadJSON()

In [7]:
# youtubeAudio.downloadMP3()

168
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
exis

existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed


existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed


existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
existed
